In [4]:
def update_docstring(name, olddoc):
    # make sure it has a docstring
    if olddoc is None:
        return None

    # new docstring
    prefix = "%s: " % name
    if len(olddoc.split(": ")) > 1:
        newdoc = prefix + olddoc.split(": ")[1]
    else:
        newdoc = prefix + olddoc

    return newdoc

In [5]:
class RewriteDocstringMeta(type):
    """Modify docstrings to be prefixed with 'classname: '.
    To do this, we intercede before the class is created and modify the
    docstrings of its attributes.
    This will not affect inherited methods, however, so we also need to
    loop through the parent classes. We cannot simply modify the
    docstrings, because then the parent classes' methods will have the
    wrong docstring. Instead, we must actually copy the functions, and
    then modify the docstring.
    """

    def __new__(cls, name, parents, attrs):
        
        # loop through each attribute in attrs.
        for attr_name in attrs:
            # skip special methods
            if attr_name.startswith("__"): # special methods start with __
                continue

            # skip non-functions. 
            #__call__ is a feature of function attributes (bc they are callable)
            attr = attrs[attr_name]
            if not hasattr(attr, '__call__'): # if doesn't have __call__, not a function.
                continue

            # update docstring
            attr.__doc__ = update_docstring(name, attr.__doc__)

        for parent in parents:
            for attr_name in dir(parent):

                # we already have this method
                if attr_name in attrs:
                    continue

                # skip special methods
                if attr_name.startswith("__"):
                    continue

                # get the original function and copy it
                a = getattr(parent, attr_name)

                # skip non-functions
                if not hasattr(a, '__call__'):
                    continue

                # copy function
                f = a.__func__
                attr = type(f)(
                    f.func_code, f.func_globals, f.func_name,
                    f.func_defaults, f.func_closure)
                doc = f.__doc__

                # update docstring and add attr
                attr.__doc__ = update_docstring(name, doc)
                attrs[attr_name] = attr

        # create the class
        # super(classname, self).attributeName()
        obj = super(RewriteDocstringMeta, cls).__new__(
            cls, name, parents, attrs)
        return obj

In [6]:
class A(object):
    __metaclass__ = RewriteDocstringMeta

    def foo(self):
        """Do some stuff."""
        pass

class B(A):
    __metaclass__ = RewriteDocstringMeta

class C(B):
    __metaclass__ = RewriteDocstringMeta

In [7]:
print A.foo.__doc__ # prints 'A: Do some stuff.'
print B.foo.__doc__ # prints 'B: Do some stuff.'
print C.foo.__doc__ # prints 'C: Do some stuff.'

A: Do some stuff.
B: Do some stuff.
C: Do some stuff.
